In [ ]:
#Functions are built for every specific purpose, and finally one function is built to call all of them for getting data for a
#given year

In [1]:
import pandas as pd
import numpy as np
import requests
from zipfile import ZipFile
from bs4 import BeautifulSoup
from pandas import Timestamp

In [2]:
pd.set_option('display.max_columns', None)

# 1. Creating 'Download' folder and downloading files into it

In [3]:
import os
c = os.getcwd()   #Get current working directory
c

'C:\\Users\\hp'

In [4]:
os.mkdir('Download')

In [5]:
cwd = c + '\\Download'
cwd

'C:\\Users\\hp\\Download'

In [6]:
os.chdir(cwd)

# 2. Downloading the zip files

In [ ]:
##Function for getting required urls for renaming files.

In [7]:
def get_url_for_download_and_rename(year, yr):
    index = 6 + yr.index(year)
    url = 'https://www.data.bsee.gov/Main/OGOR-A.aspx'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    first = soup.find('table', {'id':'ContentPlaceHolderBody_ASPxGridView1_DXMainTable', 'class':'dxgvTable'})

    req = first.find_all('tr') 

    first_td= req[index].find_all('td', class_ = 'dxgv')[0]
    l = first_td.find('a', class_="dxeHyperlink")
    l = l.get('href')

    second_td = req[index].find_all('td', class_ = 'dxgv')[1]
    a = second_td.text
    #print(a)
    a1 = a.split(" ")[0]
    a11 = a1.split("/")

    if len(a11[0]) == 1:
        a11[0] = '0' + a11[0]
    if len(a11[1]) == 1:
        a11[1] = '0' + a11[1]
    af = a11[2]+ a11[0] + a11[1]

    print(af)

    #os.rename('ogora' + str(year) + 'delimit.txt','ogora' + 'year' + '_' + af + '.txt')
    third_td = req[index].find_all('td', class_ = 'dxgv')[3]
    l1 = third_td.find('a', class_="dxeHyperlink")
    l1 = l1.get('href')
    
    urld = 'https://www.data.bsee.gov/' + l1
    
    urln = 'https://www.data.bsee.gov/' + l
    
    print(urld)
    print(urln)
    
    return af, urln, urld

In [73]:
#af, urln, urld = get_url_for_download_and_rename(year)

20220108
https://www.data.bsee.gov//Production/Files/ogora2020delimit.zip
https://www.data.bsee.gov//Main/HtmlPage.aspx?page=ogorA


In [ ]:
##Function for downloading zip files

In [8]:
def download_dataset(url):
    downloadUrl = url

    req = requests.get(downloadUrl)
    filename = req.url[downloadUrl.rfind('/')+1:]

    with open(filename, 'wb') as f:
        for chunk in req.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    
    return filename

In [74]:
#filename = download_dataset(urld)

# 3. Unzipping the files and processing

In [ ]:
##Function for extracting zip files and then removing it

In [9]:
def extract_zip_remove_zip(filename):
    
    with ZipFile(filename, 'r') as zip_object:
        zip_object.extractall()
    
    os.remove(filename)
    return

In [75]:
#extract_zip_remove_zip(filename)

In [76]:
#os.rename('ogora' + str(year) + 'delimit.txt','ogora' + str(year) + '_' + af + '.txt')

In [ ]:
##Function for obtaining the list of names of columns

In [10]:
def column_name_for_each_yr(urln):
    page = requests.get(urln)
    soup = BeautifulSoup(page.content, 'html.parser')
    t = soup.find('table')

    lc = []

    ln = t.find_all('tr')
    for i in range(1, len(ln)):
        if i != 11:
            lc.append(ln[i].find_all('td')[3].text.split("-")[0])
        else: 
            lc.append(ln[i].find_all('td')[3].find('b').text.split("-")[0])

    for i in range(len(lc)):
        lc[i] = lc[i].rstrip()
        
    return lc


In [35]:
#lc = column_name_for_each_yr(urln)
#lc

['LEASE_NUMBER',
 'COMPLETION_NAME',
 'PRODUCTION_DATE',
 'DAYS_ON_PROD',
 'PRODUCT_CODE',
 'MON_O_PROD_VOL',
 'MON_G_PROD_VOL',
 'MON_WTR_PROD_VOL',
 'API_WELL_NUMBER',
 'WELL_STAT_CD',
 'AREA_CODE',
 'OPERATOR_NUM',
 'SORT_NAME',
 'FIELD_NAME_CODE',
 'INJECTION_VOLUME',
 'PROD_INTERVAL_CD',
 'FIRST_PROD_DATE',
 'UNIT_AGT_NUMBER',
 'UNIT_ALOC_SUFFIX']

In [ ]:
##Function for creating the dataframe by importing the downloaded/extracted file and performing required operations

In [11]:
def ops_df(l, year, af):
    df = pd.read_csv('ogora' + str(year) + '_' + str(af) + '.txt', sep=",", header=None)
    df.columns = l
    df = df[[l[0], l[2], l[4], l[5], l[6], l[11]]]
    return df

In [78]:
#ops_df(lc, year)

,LEASE_NUMBER,PRODUCTION_DATE,PRODUCT_CODE,MON_O_PROD_VOL,MON_G_PROD_VOL,OPERATOR_NUM
0,00016,202001,O,0,0,3151
1,00016,202001,O,0,0,3151
2,00016,202001,O,0,0,3151
3,00016,202001,O,0,0,3151
4,00016,202001,G,0,21,3151
...,...,...,...,...,...,...
99209,S01367,202012,O,973,1009,3481
99210,S01423,202012,O,0,0,2375
99211,S01423,202012,G,0,0,2375
99212,S03839,202012,G,0,0,3151


In [ ]:
##Function for calling all the required/previously described functions

In [12]:
def final_function(year):
    yr = []
    for i in range(27):
        yr.append(2022 - i)
    index = 6 + yr.index(year)
    af, urln, urld = get_url_for_download_and_rename(year, yr)
    filename = download_dataset(urld)
    extract_zip_remove_zip(filename)
    print(index)
    print(year)
    print(cwd)
    os.rename('ogora' + str(year) + 'delimit.txt','ogora' + str(year) + '_' + af + '.txt')
    lc = column_name_for_each_yr(urln)
    df = ops_df(lc, year, af) 
    return df

In [13]:
df1= final_function(2019)
df1

20220815
https://www.data.bsee.gov//Production/Files/ogora2019delimit.zip
https://www.data.bsee.gov//Main/HtmlPage.aspx?page=ogorA
9
2019
C:\Users\hp\Download


,LEASE_NUMBER,PRODUCTION_DATE,PRODUCT_CODE,MON_O_PROD_VOL,MON_G_PROD_VOL,OPERATOR_NUM
0,00016,201901,O,0,0,3151
1,00016,201901,O,0,0,3151
2,00016,201901,O,0,0,3151
3,00016,201901,O,0,0,3151
4,00016,201901,G,0,50,3151
...,...,...,...,...,...,...
104135,S01367,201912,O,912,640,3481
104136,S01423,201912,O,1908,1158,2375
104137,S01423,201912,G,6324,3286,2375
104138,S03839,201912,G,0,0,3151


In [14]:
df2 = final_function(2020)
df2

20220815
https://www.data.bsee.gov//Production/Files/ogora2020delimit.zip
https://www.data.bsee.gov//Main/HtmlPage.aspx?page=ogorA
8
2020
C:\Users\hp\Download


,LEASE_NUMBER,PRODUCTION_DATE,PRODUCT_CODE,MON_O_PROD_VOL,MON_G_PROD_VOL,OPERATOR_NUM
0,00016,202001,O,0,0,3151
1,00016,202001,O,0,0,3151
2,00016,202001,O,0,0,3151
3,00016,202001,O,0,0,3151
4,00016,202001,G,0,21,3151
...,...,...,...,...,...,...
99209,S01367,202012,O,973,1009,3481
99210,S01423,202012,O,0,0,2375
99211,S01423,202012,G,0,0,2375
99212,S03839,202012,G,0,0,3151


In [15]:
df = pd.concat([df1, df2], axis = 0)
df

,LEASE_NUMBER,PRODUCTION_DATE,PRODUCT_CODE,MON_O_PROD_VOL,MON_G_PROD_VOL,OPERATOR_NUM
0,00016,201901,O,0,0,3151
1,00016,201901,O,0,0,3151
2,00016,201901,O,0,0,3151
3,00016,201901,O,0,0,3151
4,00016,201901,G,0,50,3151
...,...,...,...,...,...,...
99209,S01367,202012,O,973,1009,3481
99210,S01423,202012,O,0,0,2375
99211,S01423,202012,G,0,0,2375
99212,S03839,202012,G,0,0,3151


In [16]:
df.to_csv('Production.csv', header=True, index=False, sep ='|', line_terminator='\r\n')

# Basic Analysis

In [17]:
df[df['LEASE_NUMBER'].isna()]    #NaN lease rows

,LEASE_NUMBER,PRODUCTION_DATE,PRODUCT_CODE,MON_O_PROD_VOL,MON_G_PROD_VOL,OPERATOR_NUM
61211,NaN,201907,O,1947,640,3481
69852,NaN,201908,G,1272,66326,3520
78458,NaN,201909,O,463,1088,3481
50440,NaN,202006,O,112062,85602,2647


In [ ]:
#df = df.dropna(subset = 'LEASE_NUMBER')   #Depending upon requirements NaN Lease can be dropped using this formula.
                                           #commented out since it has cond=siderable production vol

In [18]:
ch = df.groupby(['LEASE_NUMBER', 'PRODUCTION_DATE', 'PRODUCT_CODE'])['MON_O_PROD_VOL', 'MON_G_PROD_VOL'].sum()
ch

C:\Users\hp\AppData\Local\Temp/ipykernel_10040/225128399.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ch = df.groupby(['LEASE_NUMBER', 'PRODUCTION_DATE', 'PRODUCT_CODE'])['MON_O_PROD_VOL', 'MON_G_PROD_VOL'].sum()


MON_O_PROD_VOL  MON_G_PROD_VOL
LEASE_NUMBER PRODUCTION_DATE PRODUCT_CODE                                
 00016       201901          G                          0              50
                             O                       8842           10385
                             U                          0               0
             201902          G                          0              20
                             O                       8445           11441
...                                                   ...             ...
S06894       202012          O                       1296               0
S19323       201901          O                          0               0
             201902          O                          0               0
             201903          O                          0               0
             201904          O                          0               0

[43887 rows x 2 columns]

# 4. Only download if the file is updated 

In [20]:
cwd

'C:\\Users\\hp\\Download'

In [ ]:
##Function for comparing timestamp of downloaded file and that available on url

In [19]:
def updated_get_url_for_download_and_rename(year, yr):
    index = 6 + yr.index(year)
    url = 'https://www.data.bsee.gov/Main/OGOR-A.aspx'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    first = soup.find('table', {'id':'ContentPlaceHolderBody_ASPxGridView1_DXMainTable', 'class':'dxgvTable'})

    req = first.find_all('tr') 

    second_td = req[index].find_all('td', class_ = 'dxgv')[1]
    a = second_td.text
    a1 = a.split(" ")[0]
    a11 = a1.split("/")

    if len(a11[0]) == 1:
        a11[0] = '0' + a11[0]
    if len(a11[1]) == 1:
        a11[1] = '0' + a11[1]
    af = a11[2]+a11[0]+a11[1]
    af = int(af)
    print(af)
    
    FOLDER_PATH = cwd
    
    l_fn = []

    def listDir(dir):
        fileNames = os.listdir(dir)
        for fileName in fileNames:
            print('filename:', fileName)
            if fileName.split(".")[-1] == 'txt':
                l_fn.append(fileName)
    
    if __name__ == '__main__':
        listDir(FOLDER_PATH)
    
    res = []

    for i in range(len(l_fn)):
        x = l_fn[i].split("_")[0]
        num = ''
        for c in x:
            if c.isdigit():
                num = num + c
        num = int(num)
        res.append(num)
        
        
    nindex = res.index(year)
    j = l_fn[nindex].split(".")[0]
    k = j.split("_")[1]
    k = int(k)
    print(k)
    
    first_td= req[index].find_all('td', class_ = 'dxgv')[0]
    l = first_td.find('a', class_="dxeHyperlink")
    l = l.get('href')

    #os.rename('ogora' + str(year) + 'delimit.txt','ogora' + 'year' + '_' + af + '.txt')
    third_td = req[index].find_all('td', class_ = 'dxgv')[3]
    l1 = third_td.find('a', class_="dxeHyperlink")
    l1 = l1.get('href')
    
    urld = 'https://www.data.bsee.gov/' + l1
    
    urln = 'https://www.data.bsee.gov/' + l
    
    print(urld)
    print(urln)
    
    return af, k, urln, urld

In [ ]:
##Function for getting the updated dataframe if required, if more recent data is present for given year in the url,
#the most recent data will be downloaded, other wise it won't do anything

In [20]:
def updated_final_function(year):
    yr = []
    for i in range(27):
        yr.append(2022 - i)
    index = 6 + yr.index(year)
    af, k, urln, urld= updated_get_url_for_download_and_rename(year, yr)
    if af > k:
        filename = download_dataset(urld)
        extract_zip_remove_zip(filename)
        print(index)
        print(year)
        print(cwd)
        os.rename('ogora' + str(year) + 'delimit.txt','ogora' + str(year) + '_' + str(af) + '.txt')
        lc = column_name_for_each_yr(urln)
        df = ops_df(lc, year, af)
    else:
        df = pd.DataFrame()
        print('Recent Data already downloaded')
    return df

In [21]:
df_u = updated_final_function(2020)

20220815
filename: ogora2019_20220810.txt
filename: ogora2020_20220715.txt
filename: Production.csv
20220715
https://www.data.bsee.gov//Production/Files/ogora2020delimit.zip
https://www.data.bsee.gov//Main/HtmlPage.aspx?page=ogorA
8
2020
C:\Users\hp\Download


In [22]:
df_u

,LEASE_NUMBER,PRODUCTION_DATE,PRODUCT_CODE,MON_O_PROD_VOL,MON_G_PROD_VOL,OPERATOR_NUM
0,00016,202001,O,0,0,3151
1,00016,202001,O,0,0,3151
2,00016,202001,O,0,0,3151
3,00016,202001,O,0,0,3151
4,00016,202001,G,0,21,3151
...,...,...,...,...,...,...
99209,S01367,202012,O,973,1009,3481
99210,S01423,202012,O,0,0,2375
99211,S01423,202012,G,0,0,2375
99212,S03839,202012,G,0,0,3151


In [23]:
df_v = updated_final_function(2019)

20220815
filename: ogora2019_20220810.txt
filename: ogora2020_20220715.txt
filename: ogora2020_20220815.txt
filename: Production.csv
20220810
https://www.data.bsee.gov//Production/Files/ogora2019delimit.zip
https://www.data.bsee.gov//Main/HtmlPage.aspx?page=ogorA
9
2019
C:\Users\hp\Download


In [24]:
df_v

,LEASE_NUMBER,PRODUCTION_DATE,PRODUCT_CODE,MON_O_PROD_VOL,MON_G_PROD_VOL,OPERATOR_NUM
0,00016,201901,O,0,0,3151
1,00016,201901,O,0,0,3151
2,00016,201901,O,0,0,3151
3,00016,201901,O,0,0,3151
4,00016,201901,G,0,50,3151
...,...,...,...,...,...,...
104135,S01367,201912,O,912,640,3481
104136,S01423,201912,O,1908,1158,2375
104137,S01423,201912,G,6324,3286,2375
104138,S03839,201912,G,0,0,3151


In [25]:
#Finally, use concat function to club dataframes together and export it as csv

In [26]:
df = pd.concat([df1, df2], axis = 0)

In [27]:
df.to_csv('Production.csv', header=True, index=False, sep ='|', line_terminator='\r\n')